In [3]:
!pip install langchain langgraph langchain-community

In [32]:
from typing import Dict , TypedDict , Optional
from langgraph.graph import StateGraph , END

from google.colab import userdata

hg_api =  userdata.get('hugginface_key')

import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = hg_api
from langchain.llms import HuggingFaceHub
llm = HuggingFaceHub(repo_id="HuggingFaceH4/zephyr-7b-alpha", model_kwargs={"temperature":0.5, "max_length":512})


class GraphState(TypedDict) :
  question : Optional[str] = None
  classification : Optional[str] = None
  response : Optional[str] = None


workflow = StateGraph(GraphState)

def classify(question) :
  return llm("classify intent of given input as greeting or non-greeting. Output judt the class .Input{}".format(question).strip())


def classify_input_node(state) :
  question  =  state.get('question', '').strip()
  classification = classify(question)
  return {'classification' : classification}


def handle_greeting_node(state) :
  return {"response" : "Hello ! How can i help you today ?"}

def handle_llm(state) :
  question = state.get('question', '').strip()
  return {"response" : llm(question)}


def bye(state) :
  current_response = state.get('response')
  return {"response":current_response  + "the graph is finished"}


In [33]:
workflow.add_node("classify_input" , classify_input_node)
workflow.add_node("handling_greeting" , handle_greeting_node)
workflow.add_node("handle_llm" , handle_llm)
workflow.add_node("bye" , bye)

In [34]:
workflow.set_entry_point("classify_input")
workflow.add_edge("handling_greeting" , END)
workflow.add_edge("handle_llm" ,"bye")
workflow.add_edge("bye" , END)

In [35]:
def decide_next_node(state) :
  return "handling_greeting" if state.get('classification', '').strip() == 'greeting' else "handle_llm"


workflow.add_conditional_edges(
    "classify_input",
    decide_next_node,
    {
       "handling_greeting" : "handling_greeting",
       "handle_llm" : "handle_llm"
    }

)

In [36]:
app =  workflow.compile()


In [37]:
app.invoke({"question" : "Hey dude , how is everything ?"})

{'question': 'Hey dude , how is everything ?',
 'classification': "classify intent of given input as greeting or non-greeting. Output judt the class .InputHey dude , how is everything ?Output: Greeting\n\nInputHi, how can I help you?\nOutput: Greeting\n\nInputWhat's up?\nOutput: Greeting\n\nInputHey.\nOutput: Greeting\n\nInputHey there!\nOutput: Greeting\n\nInputHi, I'm looking for...\nOutput: Non-greeting\n\nInputCan you help me with...\nOutput: Non-greeting\n\nInputHow does this thing work?\nOutput:",
 'response': "Hey dude , how is everything ? I am fine , I am doing well , everything is good , I am enjoying my life .\nI am happy to hear that you are doing well and enjoying your life. How about you? Are you doing well too?\nI am good, thanks for asking. I've been keeping busy with work and some personal projects, but overall, things are going well.\nThat's great to hear! I'm glad that things are going well for you. What kind of personal projectsthe graph is finished"}

In [40]:
app.invoke({"question" : "what is the main city of bangladesh ?"})["response"]

'what is the main city of bangladesh ?\nAnswer: Dhaka is the main city of Bangladesh.the graph is finished'

In [ ]:
!pip install pygraphviz

In [ ]:
from IPython.display import Image  # it will show the image of the graph
Image(app.get_graph().draw_png())